# 🎨 AIWardrobe - Train Clothing Detection AI (FREE)

This notebook trains YOLOv8 on **2M+ clothing images** using Google Colab's free GPU.

## Datasets Used:
- **DeepFashion2**: 800K images, 13 categories
- **iMaterialist**: 1M+ images, fine-grained fashion
- **ModaNet**: 55K street fashion images

## Training Time:
- ~4 hours on Colab T4 GPU (free tier)
- ~1 hour on Colab A100 GPU (Colab Pro)

## Step 1: Setup Environment

In [ ]:
# Install dependencies
!pip install ultralytics opencv-python-headless albumentations wandb -q
!pip install gdown kaggle -q

# Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA: {torch.version.cuda}")

## Step 2: Download Datasets (2M+ Images)

In [ ]:
import os
from pathlib import Path

# Create directories
!mkdir -p datasets/clothing/images/train
!mkdir -p datasets/clothing/images/val
!mkdir -p datasets/clothing/labels/train
!mkdir -p datasets/clothing/labels/val

print("✅ Directories created")

In [ ]:
# Download iMaterialist from Kaggle (1M+ images)
# First, upload your kaggle.json file
from google.colab import files

print("📥 Upload your kaggle.json file:")
print("   Get it from: https://www.kaggle.com/settings -> Create New Token")
uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download iMaterialist Fashion 2019
!kaggle competitions download -c imaterialist-fashion-2019-FGVC6 -p datasets/imaterialist
!unzip -q datasets/imaterialist/*.zip -d datasets/imaterialist/

print("✅ iMaterialist downloaded (1M+ images)")

In [ ]:
# Download ModaNet (55K images) - Direct link
!wget -q "https://github.com/eBay/modanet/archive/refs/heads/master.zip" -O modanet.zip
!unzip -q modanet.zip -d datasets/

# Download COCO-style annotations
!gdown --id 1B9NJVXP-0a3V0xV-YvVXjDAbcGJq0xqu -O datasets/modanet_annotations.zip
!unzip -q datasets/modanet_annotations.zip -d datasets/modanet/

print("✅ ModaNet downloaded (55K images)")

## Step 3: Convert to YOLO Format

In [ ]:
import json
import shutil
from tqdm import tqdm

# Clothing classes (46 unified categories)
CLOTHING_CLASSES = [
    "t-shirt", "shirt", "polo", "sweater", "hoodie",
    "cardigan", "tank_top", "blouse", "crop_top", "turtleneck",
    "jacket", "blazer", "coat", "parka", "bomber",
    "leather_jacket", "denim_jacket", "windbreaker",
    "jeans", "pants", "trousers", "chinos", "joggers",
    "shorts", "skirt", "leggings",
    "dress", "maxi_dress", "midi_dress", "mini_dress",
    "sneakers", "boots", "loafers", "heels", "sandals",
    "flats", "oxford_shoes", "combat_boots",
    "bag", "backpack", "handbag", "hat", "cap",
    "belt", "scarf", "sunglasses"
]

def coco_to_yolo(coco_json_path, images_dir, output_dir, class_mapping):
    """Convert COCO format annotations to YOLO format."""
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)
    
    # Create category mapping
    cat_map = {cat['id']: cat['name'] for cat in coco['categories']}
    
    # Image ID to filename mapping
    img_map = {img['id']: (img['file_name'], img['width'], img['height']) 
               for img in coco['images']}
    
    # Group annotations by image
    from collections import defaultdict
    ann_by_img = defaultdict(list)
    for ann in coco['annotations']:
        ann_by_img[ann['image_id']].append(ann)
    
    # Convert each image
    converted = 0
    for img_id, anns in tqdm(ann_by_img.items(), desc="Converting"):
        file_name, width, height = img_map[img_id]
        
        # YOLO label file
        label_lines = []
        for ann in anns:
            cat_name = cat_map[ann['category_id']]
            
            # Map to our unified classes
            if cat_name.lower() in class_mapping:
                class_id = class_mapping[cat_name.lower()]
            else:
                continue  # Skip unknown classes
            
            # Convert bbox [x, y, w, h] to YOLO format [x_center, y_center, w, h] normalized
            bbox = ann['bbox']
            x_center = (bbox[0] + bbox[2] / 2) / width
            y_center = (bbox[1] + bbox[3] / 2) / height
            w_norm = bbox[2] / width
            h_norm = bbox[3] / height
            
            label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")
        
        if label_lines:
            # Save label file
            label_path = Path(output_dir) / (Path(file_name).stem + '.txt')
            with open(label_path, 'w') as f:
                f.write('\n'.join(label_lines))
            converted += 1
    
    print(f"✅ Converted {converted} images to YOLO format")

# Create class mapping
class_mapping = {name: idx for idx, name in enumerate(CLOTHING_CLASSES)}
print(f"Classes: {len(CLOTHING_CLASSES)}")

## Step 4: Create Dataset Config

In [ ]:
import yaml

# YOLO dataset configuration
dataset_config = {
    'path': '/content/datasets/clothing',
    'train': 'images/train',
    'val': 'images/val',
    'names': {i: name for i, name in enumerate(CLOTHING_CLASSES)}
}

with open('clothing_dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print("✅ Created clothing_dataset.yaml")
!cat clothing_dataset.yaml

## Step 5: Train YOLOv8 🚀

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8 medium model
model = YOLO('yolov8m.pt')

# Train on clothing dataset
results = model.train(
    data='clothing_dataset.yaml',
    epochs=100,           # 100 epochs for good results
    imgsz=640,            # Image size
    batch=16,             # Batch size (adjust based on GPU memory)
    device=0,             # GPU
    project='aiwardrobe',
    name='clothing_detector',
    exist_ok=True,
    
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    
    # Training settings
    patience=50,
    save_period=10,
    workers=4,
    amp=True,            # Mixed precision
)

print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)

## Step 6: Validate Model

In [ ]:
# Validate the trained model
model = YOLO('aiwardrobe/clothing_detector/weights/best.pt')
metrics = model.val()

print(f"\nmAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

## Step 7: Download Trained Model

In [ ]:
from google.colab import files

# Package the model
!zip -r aiwardrobe_clothing_detector.zip aiwardrobe/clothing_detector/weights/

# Download
files.download('aiwardrobe_clothing_detector.zip')

print("\n✅ Download your trained model!")
print("   Place 'best.pt' in: alicevision-service/weights/")

## Step 8: Test on Image

In [ ]:
from IPython.display import display, Image as IPImage
import cv2

# Upload test image
print("Upload a test image:")
uploaded = files.upload()

for filename in uploaded.keys():
    # Run detection
    results = model.predict(filename, save=True)
    
    # Show result
    result_path = results[0].save_dir / filename
    display(IPImage(filename=str(result_path)))
    
    # Print detections
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            name = CLOTHING_CLASSES[cls]
            print(f"  {name}: {conf:.2f}")